In [1]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
dirs = ["../.."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType, CustomTemplateType

model_type = CustomModelType.llama_3_8b_instruct
llm_engine = get_vllm_engine(
    model_type, 
    # torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=1,
    # max_model_len=8192,
    # gpu_memory_utilization=1,
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 128,
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)



2024-04-20 22:48:36,953 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-04-20 22:48:36,956 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-04-20 22:48:36,983 - modelscope - INFO - Loading done! Current index file version is 1.13.3, with md5 2390e04ce69984a7eaf9207ea8d4380c and a total number of 972 components indexed
[INFO:swift] Loading the model using model_dir: /home/css/models/Meta-Llama-3-8B-Instruct
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: LlamaConfig {
  "_name_or_path": "/home/css/models/Meta-Llama-3-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_

INFO 04-20 22:48:38 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='/home/css/models/Meta-Llama-3-8B-Instruct', tokenizer='/home/css/models/Meta-Llama-3-8B-Instruct', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=42)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-20 22:48:38 selector.py:51] Cannot use FlashAttention because the package is not found. Please install it for better performance.
INFO 04-20 22:48:38 selector.py:25] Using XFormers backend.
INFO 04-20 22:48:42 model_runner.py:104] Loading model weights took 14.9595 GB
INFO 04-20 22:48:43 gpu_executor.py:94] # GPU blocks: 2601, # CPU blocks: 2048
INFO 04-20 22:48:44 model_runner.py:791] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-20 22:48:44 model_runner.py:795] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-20 22:48:47 model_runner.py:867] Graph capturing finished in 3 secs.


In [5]:
generation_config = VllmGenerationConfig(
    max_new_tokens=4096,
    temperature=0,
)

request_list = [
#     {'query': 'Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.\
# \n\nCLAIM:\nPublication date: 2020-08-13\nContent: Illinois Strengthens Face Mask Rules in Businesses\n\nINFORMATION:\nInformation 1:\nPublication date: 2013-08-06\nTitle: Illinois Strengthens Face Mask Rules in Businesses - WebMD\nContent:\nCOVID-19 is a new type of coronavirus that causes mild to severe cases. Here’s a quick guide on how to spot symptoms, risk factors, prevent spread of the disease, and find out what to do if you think you have it.\nInformation 2:\nPublication date: None\nTitle: press-release\nContent:\nIllinoisans can resume activities without wearing a mask indoors on February 28th except where required by federal, state, local, tribal, or territorial laws, rules, and regulations, including local business and workplace guidance. Federal requirements, in effect through at least March 18, include all transportation systems such as airports, planes, trains, and buses. "Preparing to repeal statewide masking mandates at the end of the month is aggressive and optimistic but reasonable," said Dr.\n"Broad mandates are not about individuals. They are put in place to help protect communities, businesses, and healthcare access. Repealing the mask mandate allows people to choose the mitigation layers that are best for them and I have no doubt that many should and will choose to keep mask rules."\nIf these trends continue — and we expect them to —then on Monday, February 28th, we will lift the indoor mask requirement for the State of Illinois," said Governor JB Pritzker. "I want to be clear: Many local jurisdictions, businesses and organizations have their own mask requirements and other mitigations that must be respected.\nSeth Trueger, a Northwestern emergency physician who is also immunocompromised. "Masking has helped slow the spread even in the face of omicron\'s transmissibility. We can and must use this time to further increase vaccination uptake & outreach, especially among children and other populations with low vaccination rates, so when the next wave comes, we will be even better prepared."\nInformation 3:\nPublication date: None\nTitle: Mask and Vaccine Requirements FAQ\'s\nContent:\nOn September 3, 2021, the Governor signed Executive Order 21-22 which requires all individuals over the age of 2 and who can medically tolerate a face covering to wear a face covering when in indoor public places. The Executive Order also requires health care workers, school personnel, higher education personnel and students, and employees and contractors of state-owned or operated congregate facilities to be fully vaccinated, as described in the Order\nInformation 4:\nPublication date: None\nTitle: FAQ for Businesses Concerning Use of Face-Coverings During COVID-19\nContent:\nA: Businesses reserve the right to refuse service to persons unable to comply with the requirement to wear a face covering, but they are required to provide a reasonable accommodation if it does not cause an undue hardship. Businesses are encouraged to inform customers there are exceptions to the requirement that all individuals must wear a mask.\nThese frequently asked questions are to provide guidance regarding the application of the face covering requirement in Executive Order 2021-10 for businesses and other places of public accommodation subject to Article 5 of the Illinois Human Rights Act, 775 ILCS 5/. A: A face covering is a mask or cloth face covering that covers your nose and mouth.\nExceptions may be made for individuals with medical conditions or disabilities that prevent them from safely wearing a face covering. For more information, refer to the questions on reasonable accommodations. A: Masks still must be worn by everyone on planes, buses, trains, and other forms of public transportation; in transportation hubs, such as airports and train and bus stations; in health care settings; and in congregate facilities, such as correctional facilities and homeless shelters.\nThese frequently asked questions are to provide guidance regarding the application of the face covering requirement in Executive Order 2021-10 for businesses and other places of public accommodation subject to Article 5 of the Illinois Human Rights Act, 775 ILCS 5/. When Face Coverings are Required Q: What does it mean to wear a face covering?\nInformation 5:\nPublication date: 2020-04-22\nTitle: Facing Your Face Mask Duties – A List of Statewide Orders | Littler ...\nContent:\n********************************************************NOTE: Given the reduction in activity on this topic,THIS POST WILL NO LONGER BE UPDATED, as of November 10, 2022.********************************************************Governors and public health officials across the country implemented stringent mitigation measures to help contain the spread of COVID-19.\n'},
    {'query': "how are you?"},
]


resp_list = inference_vllm(
    llm_engine, template, request_list, generation_config=generation_config, 
    # use_tqdm=True,
    verbose=True, prompt_prefix="", output_prefix=""
)

for request, resp in zip(request_list, resp_list):
    print(f"query:\n{request['query']}")
    print()
    print(f"response:\n{resp['response']}")


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

how are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'm just a language model, I don't have emotions or feelings like humans do, so I don't have a personal sense of well-being or mood. However, I'm functioning properly and ready to assist you with any questions or tasks you may have!<|eot_id|>
query:
how are you?

response:
I'm just a language model, I don't have emotions or feelings like humans do, so I don't have a personal sense of well-being or mood. However, I'm functioning properly and ready to assist you with any questions or tasks you may have!
